# Citizen Science Notebook
This notebook demonstrates the usage of the Butler to curate data and store it for later retrieval.

## Create a Zooniverse Account
If you haven't already, [create a Zooniverse account here.](https://www.zooniverse.org/)
After creating your account, return to this notebook.

## Terminal Prep Work
The follow cell will run the necessary terminal commands that make this notebook possible.

In [ ]:
# Install panoptes client package to dependencies
!python -m pip install panoptes-client
!mkdir -p project/citizen-science/astro-cutouts/
!mkdir -p project/citizen-science/org

In [ ]:
!python -m pip install google-cloud-storage

## Log in to Zooniverse
Now that you have a Zooniverse account, log into the Zooniverse(Panoptes) client.

In [ ]:
# Log into Zooniverse
import panoptes_client
client = panoptes_client.Panoptes.connect(login="interactive")

 ## Look Up Your Zooniverse Project
 The following code will not work if you have not authenticated in the cell titled "Log in to Zooniverse". </br>
 Supply the project name in the variable below.
 </br></br>
 Not that the `Project.find()` method expects the project name to reflect the "slug" of your project, if you don't know what a "slug" is in this context, see:</br>
 https://www.zooniverse.org/talk/18/967061?comment=1898157&page=1

In [ ]:
# Replace the empty string below with your project name
slugName = "doctor-toboggan/wizard-wesearch" # Add your slug name

project = panoptes_client.Project.find(slug=slugName)
projectId = project.id

# If the following prints out a number, then your project lookup was successful.
print(project.id)

## Import Butler/LSST Stack dependencies
Before you can curate data, you need to load all of the LSST stack dependencies in order to use the Butler service.

In [ ]:
# Generic python packages
import numpy as np
import pylab as plt
 
# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
 
# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst.geom as geom
import lsst.afw.coord as afwCoord
afwDisplay.setDefaultBackend('matplotlib')

 ## Prep Work
 Variable constants declared

In [ ]:
# This should match the verified version listed at the start of the notebook
! eups list -s lsst_distrib
 
# DP0.1 repo:
# Check with DM (RSP team) to see if the below the terms will make sense to the 
email = "" # Add your primary email
datasetId = "u/" + email + "/change-this2" # Replace "change-this" with a unique name of your change, leave the leading slash '/'
repo = 's3://butler-us-central1-dp01' # Keep track of this URI for later use with: butler retrieve-artifacts...
collection = "2.2i/runs/DP0.1"

## Initialize the Butler service
Notice the run and collections pulled in from the prep work above

In [ ]:
# Initialize the Butler
# the 'run' kwarg is an arbitrary name that must be unique, that is to say that once I use this name
# the Butler will complain if I try to rerun this code with the same value for 'run'
butler = dafButler.Butler(repo, collections=collection, run=datasetId)
registry = butler.registry
print(registry) # print just so we know something happened

And list all available collections:

In [ ]:
# Another cell above this cell that can be uncommented that will show all the collections that can be queried
for c in sorted(registry.queryCollections()):
    print(c)

## Query the Butler for data
`butler.get()` queries the object store and database and return the results in the form of a Python object

In [ ]:
# Specify the data to get
dataId = {'visit': '703697', 'detector': 80} # Hardcoded for now
calexp = butler.get('calexp', dataId=dataId)# Hardcoded for now
print(calexp) # print just so we know something happened

## Store the dataset in the IDF
`butler.put()` stores the Python object reference that contains the `butler.get()` query results in the IDF. It can then be retrieved with the Butler CLI.

In [ ]:
# First, delete the dataset if it already exists in the IDF
# runs = ("")
# runsIter = iter(runs)
# print(datasetId)
# deleted = butler.removeRuns([datasetId], True)
# print(deleted)

# Do butler.put() to store the retrieved data with a "run" to identify the stored dataset
print(datasetId)
zoonyTest = butler.put(calexp, 'calexp', dataId=dataId, run=datasetId) # Hardcoded 'calexp' for now
# from pprint import pprint
print(zoonyTest)


## Prep the Data for Zooniverse
Technically, the data will be stored in a Rubin datacenter, but this process will also inform Zooniverse of the new data for your project.

In [ ]:
import urllib.request
sourceId = "" # Add 
edcEndpoint = "https://rsp-data-exporter-e3g4rcii3q-uc.a.run.app/citizen-science-ingest?email=" + email + "&collection=" + datasetId + "&sourceId=" + sourceId + "&vendorProjectId=" + projectId # RSP user will expect to just see a function call, not the details
print('Processing data for Zooniverse, this may take up to a few minutes.')
response = urllib.request.urlopen(edcEndpoint).read()
manifestUrl = response.decode('UTF-8')
print(manifestUrl)

## Create a New Subject Set for Your Data
Zooniverse refers to each distinct batch of data that is associated to a project as a "subject set". This really just means to you what you're sending over is "data", but Zooniverse refers to it as a "subject set". It's also a useful way to keep track of each distinct batch of data as you can give each batch a distinct name.

In [ ]:
# Create a new subject set
subject_set = panoptes_client.SubjectSet()
subject_set.links.project = project

# Give the subject set a display name (that will only be visible to you on the Zooniverse platform)
subject_set.display_name = 'Large cutouts' # create a unique name for your subject set, which is how you will identify it 

subject_set.save()
project.reload()
subject_set

## Associate the Subject Set With Your Data
This step informs Zooniverse that there is a new subject set(data) available for it to pickup and associate with your project.

In [ ]:
import json
subject_set.id
payload = {"subject_set_imports": {"source_url": "https://storage.googleapis.com/citizen-science-data/4770983a-9fdb-4500-8715-0fc678cfdce9/manifest.csv", "links": {"subject_set": subject_set.id}}}

json.dumps(payload)
payload

## Notify Zooniverse of the New Subject Set
Finally, we send a request out to the Zooniverse API to inform them that new data is available and which subject set to associate it with.

In [ ]:
json_response, etag = client.post(path='/subject_set_imports', json=payload)
print(json_response)
json_response

## Check the Status of Your Data
The below cell will query the Zooniverse API to check the status of the subject set that you just notifed Zooniverse of.

In [ ]:
status = client.get(path='/subject_sets/{}'.format(subject_set.id))
print("============")
client.endpoint
print("============")
status

## Create Image Cutout from HiPS Server
You can create a cutout image of a HiPS survey/server by doing the following:

In [ ]:
# Import the necessary astro/plot libraries
from astroquery.hips2fits import hips2fits
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
import astropy.units as u
from astropy.coordinates import Longitude, Latitude, Angle

# Import organizational libraries
import time
import uuid
import os
import shutil

# Reference the EDC HiPS Survey
hips = 'https://storage.googleapis.com/hips-data/images'

# Configure the query and execute it
result = hips2fits.query(
   hips=hips,
   width=1000,
   height=500,
   ra=Longitude(0 * u.deg),
   dec=Latitude(20 * u.deg),
   fov=Angle(10 * u.deg),
   projection="AIT",
   get_query_payload=False,
   format='jpg',
   min_cut=0.5,
   max_cut=99.5,
   cmap=Colormap('viridis'),
)

# Create the plot
im = plt.imshow(result)
guid = str(uuid.uuid4())
cutoutsDir = "./project/citizen-science/astro-cutouts/"
dataDir = cutoutsDir + guid
os.mkdir(dataDir);
# plt.imsave("./project/citizen-science/astro-cutouts/" + guid + "/cutout-" + str(round(time.time() * 1000)) + "-" + str(1) + ".png", result) # commented out for testing purposes

# beginning of temporary testing code
for x in range(100): # create 100 cutouts from the one cutout image
    plt.imsave(dataDir + "/cutout-" + str(round(time.time() * 1000)) + "-" + str(x) + ".png", result)

shutil.make_archive(cutoutsDir + guid, 'zip', dataDir)
# end of temporary testing code

# Show the plot
plt.show() 

## Now Upload the Zipped Cutouts to Zooniverse

You will need a service account key in your RSP `/home/project/citizen-science/org` folder for the next cell to work as expected. Ensure that you change the name of the `service_account_key` to the name of the JSON key file.

In [ ]:
from google.cloud import storage

bucket_name = "citizen-science-data"
service_account_key = "skyviewer-398f28c943e8.json" # replace this with the GCP key provided to you
destination_blob_name = guid + ".zip"
source_file_name = cutoutsDir + guid + ".zip"

storage_client = storage.Client.from_service_account_json(
    './project/citizen-science/org/' + service_account_key)
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

blob.upload_from_filename(source_file_name)

print(
    "File {} uploaded to {}.".format(
        source_file_name, destination_blob_name
    )
)

## Notify the EDC of the New Citizen Science Cutout Data

Add your email address if you already haven't, and the `projectId` here inserted as a query param in the edcEndpoint is assigned in above in this notebook upon signing into the Zooniverse API.

In [ ]:
import urllib.request
email = "" # Add email
# vendor_project_id = ""
edcEndpoint = "https://rsp-data-exporter-e3g4rcii3q-uc.a.run.app/new-bucket-ingest?email=" + email + "&vendor_project_id=" + projectId + "&guid=" + guid
print('Processing data for Zooniverse, this may take up to a few minutes.')
response = urllib.request.urlopen(edcEndpoint).read()
manifestUrl = response.decode('UTF-8')
print(manifestUrl)